In this Jupyter Notebook there is the application of PCA over 
the "Pizza's Components" dataset.
The canonical representation of a dataset is the following.<br>
It is a datamatrix $X \in R^{n \times p}$ where: <br>
$\hspace{1cm}\bullet \hspace{0.5cm}n$ is the number of samples<br>
$\hspace{1cm}\bullet \hspace{0.5cm}p$ is the number of the features

On the rows there are samples while on the columns there are features:<br>
$\hspace{1cm}\bullet \hspace{0.5cm} x_{i} \in R^{p} \rightarrow$ each sample is a vector in a $p$-dimensional space <br>
$\hspace{1cm}\bullet \hspace{0.5cm} p_{j} \in R^{n} \rightarrow$ each feature is a vector in a $n$-dimensional space <br>


In [14]:
import funcs
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn
from IPython.display import Image

from sklearn import datasets
print("Pandas Version: {}".format(pd.__version__))
print("Numpy Version: {}".format(np.__version__))
print("Matplotlib Version: {}".format(matplotlib.__version__))
print("Scikit-learn Version: {}".format(sklearn.__version__))
print("Seaborn Version: {}".format(sns.__version__))
np.set_printoptions(suppress=True, linewidth=130)

Now it will be loaded the dataset about pizza different components

In [7]:
dataframe = pd.read_csv("D:\\Placement\\ML_Project\\dataset\\pca_pizza.csv")
n = np.shape(dataframe)[0]
p = np.shape(dataframe)[1]
print("Shape of Dataset: {}\n\t* Number of samples:\t{}\n\t* Number of features:\t{}"
      .format(np.shape(dataframe), n, p))

This is a first inspection about<br>
$\hspace{1cm}\bullet \hspace{0.5cm}$ Type of columns<br>
$\hspace{1cm}\bullet \hspace{0.5cm}$ Missing values

In [32]:
print(funcs.InfoColumns(dataframe))

These are some statistics useful to steer the whole analysis:<br>
$\hspace{1cm}\bullet \hspace{0.5cm}$ Descriptive statistics <br>
$\hspace{1cm}\bullet \hspace{0.5cm}$ Skewness <br>
$\hspace{1cm}\bullet \hspace{0.5cm}$ Kurtosis <br>


In [33]:
print(dataframe.describe(include='all'))

In [34]:
print(dataframe.drop(['brand','id'], axis=1).skew(axis=0))

In [35]:
print("\n",dataframe.drop(['brand','id'], axis=1).kurtosis(axis=0))

In [10]:
plt.figure(figsize=(10,6))
dataframe.drop(['brand','id'], axis=1).boxplot(figsize=(10,6))
plt.savefig("D:\\Placement\\ML_Project\\plots\\pca_box_plot.png")

In [11]:
sns.set(style="ticks", color_codes=True)
scatter = sns.pairplot(data=dataframe.drop(['id'], axis=1), markers='o')
scatter.savefig("D:\\Placement\\ML_Project\\plots\\scatters.png")

Now it will computed the centered version of the dataset.<br>
***
What does it means?<br>
***
In this way, it's performed a shift from the original canonical axis into the center of the distribution of the 
dataset.<br>
It's computed an approximation which allows to discuss about the linear relationships (if exist) among features

In this specific case it has been dropped the columns about brand and id

In [12]:
X = dataframe.drop(['brand', 'id'], axis=1).values

In [13]:
Xc = X - np.mean(X, axis=0)

Now there are two different strategies: <br>
***
PCA over the Covariance Matrix $C_{X_{C}}$<br> 
if the variances respect the order of importance that we want to attribute to the variables (it's defined a hierarchy 
based on relevance)<br>
PCA over the Correlation Matrix $R_{X_{C}}$ <br>
if we want to attribute the same importance to all the variables<br>

***
HOW CAN WE CHOOSE WHAT MATRIX TO USE?
***
It must be computed some statistical measures over the original dataset (this is a multivariate case, since $p>1$) and 
check what is the variance of each feature.<br>
In particular, if there is an high difference among variances, due to different measurement units, then it is 
recommended to compute the Correlation Matrix.
***
NB:: If the centered matrix $X_{C}$ is standardized, then $\implies (C_{X_{C}} = R_{X_{C}})$

Now it will be computed both the Covariance Matrix $C_{X_{C}}$ and the Correlation Matrix $R_{X_{C}}$ on the 
transpose of the centered dataset $C_{{(X)}^{T}}$, because the goal of this analysis is find, if exist, 
some relations among features and not samples.
***
(NB) Computing the Covariance matrix on the centered dataset is equivalent to compute it on the original ones



In [15]:
# Covariance Matrix Image 
Image("D:\\Placement\\ML_Project\\images\\  covariance_matrix.png")

In [40]:
Cov_Xc = np.cov(Xc.T)
print("This is the Covariance Matrix C of the Transpose Centered Dataset:\n\n{}"
      .format(np.array_str(Cov_Xc)))

In [16]:
# Covariance Matrix Image 
Image("D:\\Placement\\ML_Project\\images\\correlation_cofficient.png")

In [41]:
Corr_Xc = np.corrcoef(Xc.T)
print("This is the Correlation Matrix R of the Transpose Centered Dataset:\n\n{}"
      .format(np.array_str(Corr_Xc)))

Then it will be computed the eigenvalues and eigenvectors of both matrices.

In [42]:
eigenvalues_covariance, eigenvectors_covariance = np.linalg.eig(Cov_Xc)
print("These are the eigenvalues of Covariance Matrix C:\n{}\n".format(np.array_str(eigenvalues_covariance)))
print("These are the eigenvectors of the Covariance Matrix C:\n{}".format(np.array_str(eigenvectors_covariance)))

In [43]:
eigenvalues_correlation, eigenvectors_correlation = np.linalg.eig(Corr_Xc)
print("These are the eigenvalues of Correlation Matrix R:\n{}\n".format(np.array_str(eigenvalues_correlation)))
print("These are the eigenvectors of the Correlation Matrix R:\n{}".format(np.array_str(eigenvectors_correlation)))

The matrix of eigenvectors (for C and R) represent the rotation matrix $A_{p}$ such that:<br>
$Y = X \cdot A_{p}$ where 
\begin{cases}
\bullet \hspace{0.5cm} Y \in R^{n \times p} \rightarrow \text{ this is the matrix of scores (PC's)}\\
\bullet \hspace{0.5cm} X \in R^{n \times p}  \rightarrow \text{ this is the original matrix}\\
\bullet \hspace{0.5cm} A_{p} \in R^{p \times p}  \rightarrow \text{this is the matrix of loadings}
\end{cases} <br>
The matrix $A_{p}$ rotates original data into the direction of maximum variance of the dataset and is useful to:<br>
$\hspace{1cm} 1. \hspace{0.5cm}$ Perform a feature selection of the original variables<br>
$\hspace{1cm} 2. \hspace{0.5cm}$ Gives Interpretation of the PC's <br>
$\hspace{2.3cm} (\bullet) \hspace{0.5cm}$in terms of magnitude of absolute values of axis<br>
$\hspace{2.3cm} (\bullet) \hspace{0.5cm}$using the correlation coefficients among PC's and original features $X_{i}$<br>

### Let's consider the matrix $A_{p}$ related to the Covariance Matrix $C$
In the column 1, the max absolute value is situated in position 6.<br>
Hence, the feature 'Carboydrates' is going to be relevant for the construction of $Y_{1}$<br><br>
In the column 2, the max absolute value is situated in position 1.<br>
Hence, the feature 'Mois' is going to be relevant for the construction of $Y_{2}$<br><br>
In the column 3, the max absolute value is situated in position 2.<br>
Hence, the feature 'Fat' is going to be relevant for the construction of $Y_{3}$<br><br>
In the column 4, the max absolute value is situated in position 4.<br>
Hence, the feature 'Ash' is going to be relevant for the construction of $Y_{4}$<br><br>
In the column 5, the max absolute value is situated in position 5.<br>
Hence, the feature 'Sodium' is going to be relevant for the construction of $Y_{5}$<br><br>
In the column 6, the max absolute value is situated in position 4.<br>
Hence, the feature 'Ash' is going to be relevant for the construction of $Y_{6}$<br><br>
In the column 7, the max absolute value is situated in position 7.<br>
Hence, the feature 'Calories' is going to be relevant for the construction of $Y_{7}$
***
### Let's consider the matrix $A_{p}$ related to the Correlation Matrix $R$<br><br>
In the column 1, the max absolute value is situated in position 4.<br>
Hence, the feature 'Ash' is going to be relevant for the construction of $Y_{1}$<br><br>
In the column 2, the max absolute value is situated in position 1.<br>
Hence, the feature 'Mois' is going to be relevant for the construction of $Y_{2}$<br><br>
In the column 3, the max absolute value is situated in position 2.<br>
Hence, the feature 'Protein' is going to be relevant for the construction of $Y_{3}$<br><br>
In the column 4, the max absolute value is situated in position 4.<br>
Hence, the feature 'Ash' is going to be relevant for the construction of $Y_{4}$<br><br>
In the column 5, the max absolute value is situated in position 4.<br>
Hence, the feature 'Ash' is going to be relevant for the construction of $Y_{5}$<br><br>
In the column 6, the max absolute value is situated in position 7.<br>
Hence, the feature 'Calories' is going to be relevant for the construction of $Y_{6}$<br><br>
In the column 7, the max absolute value is situated in position 6.<br>
Hence, the feature 'Carboydrates' is going to be relevant for the construction of $Y_{7}$

Now it can be computed the matrix $Y$ (hence, the principal components), and it can be defined also a semantic for 
the new features (PC's) through by the study of correlations among original features $X_{i}$ and 
principal components $Y_{j}$.

In [44]:
YC = Xc.dot(eigenvectors_covariance)
print("This is the dimension of the Y matrix \t {}\n\t(*) using the loadings of the covariance matrix C\n"
      "\t(*) It must be equal to the dimension of the original dataset\n".format(np.shape(YC)))

In [45]:
YR = Xc.dot(eigenvectors_correlation)
print("This is the dimension of the Y matrix \t {}\n\t(*) using the loadings of the correlation matrix R\n"
      "\t(*) It must be equal to the dimension of the original dataset".format(np.shape(YR)))

Now it's shown the Scree-Plot, useful to choose what is the number of components $(k)$
that is better to retained in order to account for most of the variation in the dataset. <br>
The number $k$ has been computed using the Cumulative Percentage of Total Variation.<br> 
$\hspace{1cm}(\bullet)\hspace{0.3cm}C \rightarrow t_{k} = 100 \cdot \frac{\sum_{i=1}^{k}\lambda_{i}}{\sum_{i=1}^{p}\lambda_{i}} = 
100 \cdot \frac{\sum_{i=1}^{k}\lambda_{i}}{trace(C_{X})}$<br><br>

$\hspace{1cm}(\bullet)\hspace{0.3cm}R \rightarrow t_{k} = 100 \cdot \frac{\sum_{i=1}^{k}\lambda_{i}}{p}$<br><br>

In [46]:
total_variation_covariance = np.sum(eigenvalues_covariance)
explained_variance_covariance = np.asarray(
    [100*(i/total_variation_covariance) for i in sorted(eigenvalues_covariance, reverse=True)])
print("This is the explained variance of each feature (covariance):\n\t{}"
      .format(np.array_str(explained_variance_covariance, precision=2)))
cumulative_covariance = np.cumsum(explained_variance_covariance)
print("This is the cumulative variance (covariance):\n\t{}"
      .format(np.array_str(cumulative_covariance, precision=2)))

fig1 = plt.figure(1, figsize=(10,6))
plt.title("Scree plots of Covariance Matrix")
plt.bar(x=np.arange(np.shape(explained_variance_covariance)[0]), 
        height=explained_variance_covariance, 
        width=0.4, color="green")
plt.plot(np.arange(np.shape(explained_variance_covariance)[0]), 
         explained_variance_covariance, 
         linestyle="--", marker="o", markersize=15,
         color="red", label="explained variance (covariance)")
plt.savefig("images/screeplot_covariance_pizza.png")
plt.legend()
plt.grid()
plt.show()

In [47]:
total_variation_correlation = np.sum(eigenvalues_correlation)
explained_variance_correlation = np.asarray(
    [100*(i/total_variation_correlation) for i in sorted(eigenvalues_correlation, reverse=True)])
print("This is the explained variance of each feature (correlation):\n\t{}"
      .format(np.array_str(explained_variance_correlation, precision=2)))
cumulative_correlation = np.cumsum(explained_variance_correlation)
print("This is the cumulative variance (correlation):\n\t{}"
      .format(np.array_str(cumulative_correlation, precision=2)))

fig2 = plt.figure(2, figsize=(10,6))
plt.title("Scree plots of Correlation Matrices")
plt.bar(x=np.arange(np.shape(explained_variance_correlation)[0]), 
        height=explained_variance_correlation, 
        width=0.4, color="green")
plt.plot(np.arange(np.shape(explained_variance_correlation)[0]), 
         explained_variance_correlation, 
         linestyle="--", marker="o", markersize=15,
         color="red", label="explained variance correlation")
plt.savefig("images/screeplot_correlation_pizza.png")
plt.legend()
plt.grid()
plt.show()

In [48]:
number_k = 2

### Correlation circle (Original features and PC's of the covariance matrix C)

In [49]:
CC_Covariance = []
for i in range(np.shape(X)[1]):
    f = []
    for j in range(np.shape(YC)[1]):
        c = (np.corrcoef(X[:,i], YC[:,j])[0])[1]
        f.append(c)
    CC_Covariance.append(f)

In [50]:
CorrelationCircle_Covariance = np.asarray(CC_Covariance)

In [51]:
print("This is the full Correlation Matrix (based on C):\n{}"
      .format(CorrelationCircle_Covariance))

In [52]:
print("This is the Correlation Matrix with {} PC's (based on C):\n{}"
      .format(number_k, CorrelationCircle_Covariance[:,0:number_k]))

In [53]:
fig3 = plt.figure(figsize=(8,8))
plt.title("Correlation Cirle \nFeatures vs PC's (Covariance matrix C)")
plt.hlines(y=0, xmin=-1, xmax=1, colors='black', linewidth=4, linestyle='solid')
plt.vlines(x=0, ymin=-1, ymax=1, colors='black', linewidth=4, linestyle='solid')
for i in range(0,np.shape(X)[1]):
    plt.scatter(CorrelationCircle_Covariance[i:i+1,0:1], 
                CorrelationCircle_Covariance[i:i+1,1:2], s=300, label=dataframe.columns[i+2])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.xlim(-1.1,1.1)
plt.ylim(-1.1,1.1)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), 
           fancybox=True, ncol=np.shape(X)[1])
plt.grid()
plt.tight_layout()
plt.savefig("images/circle_correlation_covariance_pizza.png")
plt.show()

### Correlation circle (Original features and PC's of the correlation matrix R)

In [54]:
CC_Correlation = []
for i in range(np.shape(X)[1]):
    f = []
    for j in range(np.shape(YR)[1]):
        c = (np.corrcoef(X[:,i], YR[:,j])[0])[1]
        f.append(c)
    CC_Correlation.append(f)

In [55]:
CorrelationCircle_Correlation = np.asarray(CC_Correlation)

In [56]:
print("This is the full Correlation Matrix (based on R):\n{}"
      .format(CorrelationCircle_Correlation))

In [57]:
print("This is the Correlation Matrix with {} PC's (based on R):\n{}"
      .format(number_k, CorrelationCircle_Correlation[:,0:number_k]))

In [58]:
fig4 = plt.figure(figsize=(8,8))
plt.title("Correlation Cirle \nFeatures vs PC's (Correlation matrix R)")
plt.hlines(y=0, xmin=-1, xmax=1, colors='black', linewidth=4, linestyle='solid')
plt.vlines(x=0, ymin=-1, ymax=1, colors='black', linewidth=4, linestyle='solid')
for i in range(0,np.shape(X)[1]):
    plt.scatter(CorrelationCircle_Correlation[i:i+1,0:1], 
                CorrelationCircle_Correlation[i:i+1,1:2], 
                s=300, label=dataframe.columns[i+2])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.xlim(-1.1,1.1)
plt.ylim(-1.1,1.1)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), 
           fancybox=True,ncol=np.shape(X)[1])
plt.grid()
plt.tight_layout()
plt.savefig("images/circle_correlation_correlation_pizza.png")
plt.show()

# Conclusion

### Covariance Matrix 
Using the Covariance matrix $C_{X}$ to perform PCA and choose a $k = 2$ (qualitative analysis) which is the number 
of components that retains $\sim 97\%$ of total information embedded in the dataset, it can be found that:<br>
$\hspace{1cm}(\bullet)\hspace{0.3cm}$ PC1 is an index of energy supply (Protein(2) + Ash(4) + Carbohydrates(6))<br>
$\hspace{1cm}(\bullet)\hspace{0.3cm}$ PC2 is an index of the heaviness of pizza (Calories(7))

### Correlation Matrix 
Using the Correlation matrix $R_{X}$ to perform PCA and choose a $k = 2$ (qualitative analysis) which is the number 
of components that retains $\sim 92\%$ of total information embedded in the dataset, it can be found that:<br>
$\hspace{1cm}(\bullet)\hspace{0.3cm}$ PC1 is an index of energy supply (Protein(2) + Ash(4) + Carbohydrates(6))<br>
$\hspace{1cm}(\bullet)\hspace{0.3cm}$ PC2 is an index of tastiness of dough's consistency (mois(1))
